In [3]:
%pwd

'e:\\Rag-gen-ai-project\\Travel-chatbot'

In [2]:
import os 
os.chdir("../")

In [4]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Function to load files [pdf] from path and extract data into document
def load_pdf_file(data): 
    loader=DirectoryLoader(
        data, 
        glob="*.pdf", 
        loader_cls=PyPDFLoader
    )
    documents=loader.load()
    return documents 

# Function take extracted data and create text chunk
def text_splitter(data): 
    text_split=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunk=text_split.split_documents(data)
    return text_chunk

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings 

In [ ]:
# function download embeddings from Huggingface 
def download_embeddings(): 
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
extracted_data=load_pdf_file("Data/")
text_chunk=text_splitter(extracted_data)
embeddings=download_embeddings()

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_19080\2168308362.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\Mayur kadam\.conda\envs\chatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from langchain.vectorstores import Chroma 

# vector database directory
directory="vector_data"

# creating vector data at defined path by using huggingface embedding model
vector_database=Chroma.from_documents(
    documents=text_chunk,
    persist_directory=directory, 
    embedding=embeddings
)

# load vector data from vector database
vector_data=Chroma(
    persist_directory=directory,
    embedding_function=embeddings
)

C:\Users\Mayur kadam\AppData\Local\Temp\ipykernel_19080\1433480664.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vector_data=Chroma(


In [10]:
retrival=vector_data.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [11]:
result=retrival.invoke("kochi")
print(result)

[Document(metadata={'source': 'Data\\travel_book.pdf', 'page': 20}, page_content="KOCHI\nPERIYAR\nMUNNAR, the most cosmopolitan city of Kerala, is also known as Queen \nof the Arabian Sea'. The distinctive architecture seen in Dutch \nmansions, Portuguese churches, Jewish synagogue, British store \nhouses and 14th century Chinese fishing nets, reflects Kochi's long \nhistory as an important sea port and trading centre. Kochi is renowned \nfor its spectacular Kathakali performance, one of the oldest theatre \nforms in India. Performed by characters with vividly painted faces and"), Document(metadata={'page': 20, 'source': 'Data\\travel_book.pdf'}, page_content='Kerala : Gods Own Country \nLocated along south western coast of India, Kerala resembles a \nbanana leaf clinging between the Western Ghats and beaches of \nArabian sea. \nKerala has been included in the top 50 tourist destinations for once \nin a life time visit by National Geographic Channel. The beauty of \nKerala lies in its 

In [ ]:
from langchain_community.llms import Ollama 
from langchain_core.prompts import ChatPromptTemplate 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain 

# using gemma3 as llm model
llm=Ollama(model="gemma3")


system_prompt=(
    "you are travel question answer ai assistant "
    "you are polite and friendly "
    "answer should be in three sentences maximum and concise"
    "\n\n"
    "{context}"
)

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

# creating llm chain
question_answer_chain=create_stuff_documents_chain(llm=llm,prompt=prompt)
rag_chain=create_retrieval_chain(retrival,question_answer_chain)

In [14]:
response=rag_chain.invoke({"input":"top place visit in kochi ?"})
print(response["answer"])

Kochi boasts several incredible attractions! The iconic Chinese Fishing Nets are a must-see, along with the historic Dutch Mansions and Portuguese churches. For a cultural experience, don’t miss a captivating Kathakali performance.
